In [2]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow as tf
from pandas import Series
from sklearn.preprocessing import MinMaxScaler


In [191]:
# load targets and convert logs to classification states = ( 1, 0, -1 ) 

df = pd.read_csv(f'../../db/data/merge/secondary/logs_.csv', index_col=0)
df.reset_index(inplace=True, drop=True)

# converting logs to 3 states = ( 1, 0, -1 ) 
def condition(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0

for sym in df.columns:
    df[sym] = df[sym].map( lambda x: condition(x) )

df = df[1001:-50000]

df.reset_index(inplace=True, drop=True)

print(df)

y = []
for i in range(len(df)):
    y.append(np.array(df.iloc[i].values))

y = np.array(y)

print(y)

       AUD_CAD  AUD_CHF  AUD_JPY  AUD_NZD  AUD_USD  CAD_CHF  CAD_JPY  CHF_JPY  \
0           -1       -1       -1       -1       -1        0       -1       -1   
1           -1        1        1        1        1        1        1        1   
2            1        1        1        1        1       -1       -1        0   
3           -1       -1       -1       -1       -1        1        1        1   
4            1        1        1        1        1       -1       -1       -1   
...        ...      ...      ...      ...      ...      ...      ...      ...   
60090        1        1        1        1       -1        1        1       -1   
60091       -1       -1       -1       -1       -1       -1       -1       -1   
60092        1       -1        1       -1        1       -1        1        1   
60093       -1        1       -1        1       -1        1        1       -1   
60094       -1       -1       -1       -1       -1        1        1       -1   

       EUR_AUD  EUR_CAD  ..

In [182]:
df = pd.read_csv('../../db/data/merge/tendency/tendency.csv', index_col=0)

df.reset_index(inplace=True, drop=True)

df.replace([np.inf, -np.inf], 0, inplace=True)

df = df[1000:-50001]

X = []
for i in range(len(df)):
    X.append(np.array(df.iloc[i].values))

X = np.array(X)

X.shape

(60095, 76)

In [192]:
X_train = X[:int(len(X)*0.5)]
y_train = y[:int(len(X)*0.5)]

X_train

array([[-1.46503661e-01, -2.00991919e-01, -2.31721550e-01, ...,
         6.23489802e-01, -8.66025404e-01, -5.00000000e-01],
       [-9.22769334e-03, -1.96839631e-02, -8.55066320e-02, ...,
         6.23489802e-01, -9.65925826e-01, -2.58819045e-01],
       [-3.89707622e-02,  1.28968814e-01,  1.46520173e-01, ...,
         6.23489802e-01, -1.00000000e+00, -1.83697020e-16],
       ...,
       [ 1.02545867e-01,  1.48836418e-02, -7.32436329e-02, ...,
         6.23489802e-01,  1.22464680e-16, -1.00000000e+00],
       [ 3.36183902e-01, -6.59328997e-02, -4.25152078e-01, ...,
         6.23489802e-01, -2.58819045e-01, -9.65925826e-01],
       [ 2.70971643e-02, -5.00125042e-02,  0.00000000e+00, ...,
         6.23489802e-01, -5.00000000e-01, -8.66025404e-01]])

In [136]:
# scaling features

def scale ( data ):

    series = Series(data)
    # prepare data for normalization
    values = series.values
    values = values.reshape((len(values), 1))
    # train the normalization
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(values)
    #print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
    # normalize the dataset and print
    normalized = scaler.transform(values)
    #print(normalized)
    # inverse transform and print
    inversed = scaler.inverse_transform(normalized)
    #print(inversed)

    return normalized, inversed


y.columns

Index(['AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 'CAD_CHF',
       'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 'EUR_CHF', 'EUR_GBP',
       'EUR_JPY', 'EUR_NZD', 'EUR_USD', 'GBP_AUD', 'GBP_CAD', 'GBP_CHF',
       'GBP_JPY', 'GBP_NZD', 'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY',
       'NZD_USD', 'USD_CAD', 'USD_CHF', 'USD_JPY'],
      dtype='object')

In [166]:

X__ = []
for i in X.columns:
    normalized, _ = scale( X_train[i] )
    X__.append(normalized)

X__ = np.array(X__)
X__ = X__.reshape(4027848, 1)

X__



array([[ 0.0988461 ],
       [ 0.10489588],
       [ 0.05442634],
       ...,
       [-0.25881905],
       [-0.5       ],
       [-0.70710678]])

In [169]:
y__ = []
for i in y.columns:
    normalized, _ = scale( y_train[i] )
    y__.append(normalized)

y__ = np.array(y__)
y__ = y__.reshape(1483944, 1)

y__

array([[-1.],
       [ 0.],
       [-1.],
       ...,
       [-1.],
       [ 1.],
       [-1.]])

In [193]:

EPOCHS = 200 

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[0])))
model.add(Dense(28, activation='softmax',))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fit model
model.fit(X_train, y_train, epochs=EPOCHS)

Epoch 1/200


ValueError: in user code:

    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 214, in assert_input_compatibility
        ra

In [196]:

import torch.utils.data as data_utils

target = pd.DataFrame(df['AUD_CAD'])

train = data_utils.TensorDataset(df, target)

train_loader = data_utils.DataLoader(train, batch_size=10, shuffle=True)

TypeError: 'numpy.int64' object is not callable

In [20]:
n_lag = 1
n_neurons = 200
n_epochs = 1000
n_batch = 1
n_neurons = 1

# design network
model = Sequential()

model.add(LSTM(n_neurons, stateful=True))

model.add(Dense(y.shape[1]))

model.compile(optimizer='adam', loss='mse', metrics='accuracy')


MAX_EPOCHS = 10000

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.Accuracy()])

  history = model.fit(X=X__['train'], y=y__['train'], epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  
  return history



In [26]:
history = model.fit(x=X__['train'], y=y__['train'], epochs=MAX_EPOCHS, validation_data=X__['valid'])
  

ValueError: Data cardinality is ambiguous:
  x sizes: 68
  y sizes: 77767
Make sure all arrays contain the same number of samples.

In [21]:
compile_and_fit(model, 

SyntaxError: unexpected EOF while parsing (<ipython-input-21-b065d77e4dba>, line 1)

In [61]:
model.fit(x=X_['train'], y=y_['train'])

ValueError: in user code:

    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/quantium/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 214, in assert_input_compatibility
        ra